In [1]:
#     =====================================================================================   #
#                                       FIRST SECTION CODE                                    #
#     =====================================================================================   #

# IMPORTING IMPORTANT MODULE..... 
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd

# IMPORTING ORIGINAL DATA 
jamb_data = pd.read_csv('Newrecord/kwasufile.csv')

# GETTING ATTRIBUTE FROM ORIGINAL DATA
candidate_name = jamb_data["RG_CANDNAME"].values
candidate_reg = jamb_data["RG_NUM"].values
names = list(candidate_name)
regs = list(candidate_reg)

# print(names)
# print(regs)

# GENERATING NEW DATAFRAME
gen_df = pd.DataFrame()
    
try:
    gen_df = pd.read_csv('Newrecord/kwasu_gen/kwasufile_gen5.csv', index=False)
except Exception: 
    pass

def create_dataframe(dataframe,  add=''):
    attributes = []

    for attr in dataframe.columns: 
        attributes.append(attr)
    if add != '':
        attributes.append(add)

    df = pd.DataFrame([] , columns=attributes)

    return df , attributes


if gen_df.empty:
    print('original data is empty')
    gen_df , _ = create_dataframe(jamb_data, 'Contact')

else: 
    pass


original data is empty


In [2]:
#     =====================================================================================   #
#                                HELPER METHOD FOR SECTION TWO                                #
#     =====================================================================================   #




# helper method to valide content compatibility 

def check_content_matching(o_reg='data_reg' , o_name='d_name' , m_reg='mine_reg' , m_name = 'm_name'): 
    
    #   : o_reg / the import jamb student registration number 
    #   : o_name / the import jamb student name 
    #   : m_reg / the mined student registration number 
    #   : m_name / the mined student name 

    validate = False 
    if (m_reg.upper().strip() == o_reg.upper().strip() and m_name.lower() in o_name.lower().split()):
        validate = True
    else : 
        validate = False
    return validate ; 

# helper method to had new row to the new table
def add_row(old_table , new_table, tel='',  m_reg = ''):
    
    if m_reg != '':
        d = old_table[old_table[old_table.columns[0]] == m_reg]
        old_val = d.values
#         print(old_val.flatten())
        temp_value = []
        for value in list(old_val.flatten()):
            temp_value.append(value)

#         print('TEL:' , tel)

        if tel != '':
            temp_value.append(str(tel))
            
            print(temp_value)
            new_table.loc[len(new_table)] = temp_value
            
            new_table.to_csv('Newrecord/kwasu_gen/kwasufile_gen5.csv')
            print('Row addes successfully.....')

            return new_table
                
# add_row(jamb_data, gen_df , tel, reg_number)


In [6]:
#     =====================================================================================   #
#                                       SECOND SECTION CODE                                   #
#     =====================================================================================   #

# eporting to json 

#  to json ...
import json as j




# NAVIGATING THE WEB BROWSER USING THE STUDENT REGISTRATION NUMBER

driver= webdriver.Chrome()

def fetch_info(reg_number):
    driver.get('https://efacility.jamb.gov.ng/PrintExaminationSlip') 
    reg = driver.find_element('id' , "txtRegNumber")
    reg.send_keys(reg_number)

    # submitting form
    # reg.submit()
    sb = driver.find_element('id' , "lnkSearch")

    sb.click()
    # driver.refresh()
    # print('WEB PAGE ', driver.current_url)


    # getting data from html document
    print('fetching content')
    url = f'https://efacility.jamb.gov.ng/TempFiles/Main_Examination_Slip_{reg_number}.htm'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    data = soup.find_all('label')

    return data , len(data)


#  scraping through web for contact 

for index in range(15209  , len(regs)):
    reg = regs[index]
    name = names[index]
      
    
#         with  open('reg_cache.json','w') as file:  
#             file.dump({'last_reg':reg ,  'last_name':name , 'index':index})

    print()
    print(f'========== SOURCE FILE SIZE IS ==>  {jamb_data.shape[0]}')
    print('===================================================')
    print(f'   START MINING   FOR :  {reg} : index : {index}  ')
    print('===================================================')

    data , lenght = fetch_info(reg)
    if lenght != 0 :
            
        # printing data for confirmation
        try:
            surname = data[1].get_text()
            firstname = data[2].get_text()
            othername = data[4].get_text()
            tel = data[12].get_text()
            reg_number = data[15].get_text()

            print('Surname : ' , surname)
            print('Fist Name : ', firstname)
            print('Other Name : ',  othername)
            print('Telephone : ' , tel)
            print('Registion Number : ' , reg_number)


        

            valid = check_content_matching(reg, name , reg_number, surname)
            print('VALID : ', valid)

            if valid :
                print('                     STUDENT CONFIRMED... ')
                add_row(jamb_data, gen_df , tel, reg_number)
                
        except Exception:
            print('error')
                  
    print('==================================================')
    print(f'              END MINING     No.   {index}       ')
    print('==================================================')
        
        
    
            
            

        
print('==================================================')
print(f'               FINAL END OF MINING               ')
print('==================================================')
        


========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202211318591GF : index : 15209  
fetching content
Surname :  Olatuja
Fist Name :  Eyiwumi
Other Name :  Oluwapelumi
Telephone :  07058488683
Registion Number :  202211318591GF
VALID :  True
                     STUDENT CONFIRMED... 
['202211318591GF', 'Olatuja Eyiwumi Oluwapelumi', 'F', 'ONDO', 136, 'Mass Communication', nan, 'Not Admitted', 'ONDO', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'IRELE', '07058488683']
Row addes successfully.....
              END MINING     No.   15209       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202210776934JA : index : 15210  
fetching content
Surname :  Agba
Fist Name :  Damaris
Other Name :  Mwuese
Telephone :  09161212607
Registion Number :  202210776934JA
VALID :  True
                     STUDENT CONFIRMED... 
['202210776934JA', 'Agba Damaris Mwuese', 'F', 'BENUE', 136, 'Economics', nan, 'Not Admitted', 'BENUE', 'KWARA STATE UNIVERSITY, MAL

fetching content
Surname :  Olatunji
Fist Name :  David
Other Name :  Leke
Telephone :  08132902759
Registion Number :  202211483969FF
VALID :  True
                     STUDENT CONFIRMED... 
['202211483969FF', 'Olatunji David Leke', 'M', 'KOGI', 136, 'Public Administration', nan, 'Not Admitted', 'KOGI', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'MOPA-MORO-MOPI', '08132902759']
Row addes successfully.....
              END MINING     No.   15221       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202211052490JF : index : 15222  
fetching content
Surname :  Opawole
Fist Name :  Faruq
Other Name :  Opeyemi
Telephone :  07059116499
Registion Number :  202211052490JF
VALID :  True
                     STUDENT CONFIRMED... 
['202211052490JF', 'Opawole Faruq Opeyemi', 'M', 'OYO', 136, 'Agriculture', nan, 'Not Admitted', 'OYO', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'OLUYOLE', '07059116499']
Row addes successfully.....
              END MINI

fetching content
Surname :  Akinfenwa
Fist Name :  Oluwabamise
Other Name :  Olaoniye
Telephone :  09159464165
Registion Number :  202210798714GA
VALID :  True
                     STUDENT CONFIRMED... 
['202210798714GA', 'Akinfenwa Oluwabamise Olaoniye', 'M', 'OGUN', 136, 'Political Science', nan, 'Not Admitted', 'OGUN', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ABEOKUTA-NORTH', '09159464165']
Row addes successfully.....
              END MINING     No.   15232       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202210885492DF : index : 15233  
fetching content
Surname :  Tajudeen
Fist Name :  Mariam
Other Name :  Babaita
Telephone :  09077403624
Registion Number :  202210885492DF
VALID :  True
                     STUDENT CONFIRMED... 
['202210885492DF', 'Tajudeen Mariam Babaita', 'F', 'KWARA', 136, 'Biochemistry', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ILORIN-WEST', '09077403624']
Row addes successfu

fetching content
Surname :  Adekunle
Fist Name :  Ramotalahi
Other Name :  Opeyemi
Telephone :  08118775943
Registion Number :  202210484941JF
VALID :  True
                     STUDENT CONFIRMED... 
['202210484941JF', 'Adekunle Ramotalahi Opeyemi', 'F', 'OYO', 135, 'Business Education', nan, 'Not Admitted', 'OYO', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'AFIJIO', '08118775943']
Row addes successfully.....
              END MINING     No.   15243       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202211329778IF : index : 15244  
fetching content
Surname :  Nasirudeen
Fist Name :  Muhammad
Other Name :  Gbolagade
Telephone :  08122581802
Registion Number :  202211329778IF
VALID :  True
                     STUDENT CONFIRMED... 
['202211329778IF', 'Nasirudeen Muhammad Gbolagade', 'M', 'OYO', 135, 'Political Science', nan, 'Not Admitted', 'OYO', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ATIBA', '08122581802']
Row addes successfully.....

fetching content
Surname :  Alaye
Fist Name :  Mujeeb
Other Name :  Mubarak
Telephone :  09134864352
Registion Number :  202210575890CA
VALID :  True
                     STUDENT CONFIRMED... 
['202210575890CA', 'Alaye Mujeeb Mubarak', 'M', 'KWARA', 134, 'Biochemistry', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ILORIN-WEST', '09134864352']
Row addes successfully.....
              END MINING     No.   15255       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202210808384GF : index : 15256  
fetching content
Surname :  Adewusi
Fist Name :  Khabirat
Other Name :  Motunrayo
Telephone :  09153046902
Registion Number :  202210808384GF
VALID :  True
                     STUDENT CONFIRMED... 
['202210808384GF', 'Adewusi Khabirat Motunrayo', 'F', 'OYO', 134, 'Banking & Finance', nan, 'Not Admitted', 'OYO', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'IBADAN NORTH EAST', '09153046902']
Row addes successfully.....
    

...

fetching content
Surname :  Ademola
Fist Name :  Dorcas
Other Name :  Dada
Telephone :  09049804039
Registion Number :  202210365770BF
VALID :  True
                     STUDENT CONFIRMED... 
['202210365770BF', 'Ademola Dorcas Dada', 'F', 'KWARA', 130, 'Mass Communication', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ISIN', '09049804039']
Row addes successfully.....
              END MINING     No.   15310       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202210203819IF : index : 15311  
fetching content
Surname :  Bashiru
Fist Name :  Aliyu
Other Name :  Olaide
Telephone :  08118986418
Registion Number :  202210203819IF
VALID :  True
                     STUDENT CONFIRMED... 
['202210203819IF', 'Bashiru Aliyu Olaide', 'M', 'KWARA', 130, 'Political Science', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'IFELODUN', '08118986418']
Row addes successfully.....
              END MINING

fetching content
Surname :  Lukman
Fist Name :  Zulaikah
Other Name :  Taye
Telephone :  08166939902
Registion Number :  202210530502AF
VALID :  True
                     STUDENT CONFIRMED... 
['202210530502AF', 'Lukman Zulaikah Taye', 'F', 'KWARA', 128, 'Entrepreneurship', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ASA', '08166939902']
Row addes successfully.....
              END MINING     No.   15321       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202211612140HA : index : 15322  
fetching content
Surname :  Sanni
Fist Name :  Habeeb
Other Name :  Babatunde
Telephone :  09037382688
Registion Number :  202211612140HA
VALID :  True
                     STUDENT CONFIRMED... 
['202211612140HA', 'Sanni Habeeb Babatunde', 'M', 'KWARA', 128, 'Mass Communication', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'IREPODUN', '09037382688']
Row addes successfully.....
              END MI

fetching content
Surname :  Odofin
Fist Name :  Mistura
Other Name :  Ayoola
Telephone :  07043807413
Registion Number :  202210784247AF
VALID :  True
                     STUDENT CONFIRMED... 
['202210784247AF', 'Odofin Mistura Ayoola', 'F', 'KWARA', 125, 'Performing Arts', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'IFELODUN', '07043807413']
Row addes successfully.....
              END MINING     No.   15332       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202211024878CF : index : 15333  
fetching content
Surname :  Abdrasheed
Fist Name :  Sherif
Other Name :  Olamilekan
Telephone :  08162336846
Registion Number :  202211024878CF
VALID :  True
                     STUDENT CONFIRMED... 
['202211024878CF', 'Abdrasheed Sherif Olamilekan', 'M', 'OYO', 123, 'Mass Communication', nan, 'Not Admitted', 'OYO', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ATIBA', '08162336846']
Row addes successfully.....
         

fetching content
Surname :  Babajide
Fist Name :  Aseyori
Other Name :  Deborah
Telephone :  09157510004
Registion Number :  202210573044JF
VALID :  True
                     STUDENT CONFIRMED... 
['202210573044JF', 'Babajide Aseyori Deborah', 'F', 'KWARA', 117, 'Accounting', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'IREPODUN', '09157510004']
Row addes successfully.....
              END MINING     No.   15343       

========== SOURCE FILE SIZE IS ==>  15355
   START MINING   FOR :  202211609518HA : index : 15344  
fetching content
Surname :  Yussuf
Fist Name :  Ridwan
Other Name :  Owolabi
Telephone :  09060453237
Registion Number :  202211609518HA
VALID :  True
                     STUDENT CONFIRMED... 
['202211609518HA', 'Yussuf Ridwan Owolabi', 'M', 'KWARA', 116, 'Mass Communication', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'IFELODUN', '09060453237']
Row addes successfully.....
              E

fetching content
Surname :  Nuru
Fist Name :  Hakeem
Other Name :  Kehinde
Telephone :  08068076686
Registion Number :  202210186242JF
VALID :  True
                     STUDENT CONFIRMED... 
['202210186242JF', 'Nuru Hakeem Kehinde', 'M', 'KWARA', 105, 'Business Education', nan, 'Not Admitted', 'KWARA', 'KWARA STATE UNIVERSITY, MALETE, ILORIN, KWARA STATE', 'ILORIN-WEST', '08068076686']
Row addes successfully.....
              END MINING     No.   15354       
               FINAL END OF MINING               


In [ ]:
# 4331  